所学：在本节中，学会了keras进行文本分类的一个流程思路，以及如何搭建的
     在里面把训练集、验证集、测试集三者的关系搞清楚了
疑惑点：网络搭建的时候为什么用那种层，原理和依据是什么

与图片数据相比，文本数据有以下几个特点：

- 长度不确定；
- 语言之间的差异较大，编码方式各不相同；
- 同一种语言的处理方式也不尽相同；
- 特征提取方式不统一。

因为文本数据的不确定性，因此我们这节课采用最常用的数据处理方式（单词嵌入）与最常用的文本分类数据集（ IMBD® 评价数据集）。

1.数据集合概览
   IMDB® 数据集合一共包含 50000 条数据，每条数据都是从 IMDB® 电影的评价中选取，同时每个评论都被归类为“正面评价”或“负面评价”。
   其中评论是被编码之后所得到的数组，每个英文单词对应一个固定的数字。而标签用 0 和 1 来表示“负面评价”和“证明评价”。
   这 50000 条数据它们具体的分布如下：
   训练集包含 25000 条训练数据，其中正负数据各 12500 条；
   测试集包含 25000 条测试数据，其中正负数据各 12500 条。
   该数据集合上面的数据是“平衡的”，因为它包含的正样本与负样本的数目相同。
   在 TensorFlow 之中，我们可以直接通过调用内部 API 的方式来获取该数据集：
   
   

In [ ]:
# 在这里反斜杠是起到转义的作用:字符串中的引号进行转义
(train_data, train_labels), (test_data, test_labels) = \
tf.keras.datasets.imdb.load_data(num_words=words_num)


2. 如何对文本数据进行处理
在机器学习之中，我们对于文本数据的处理大致分为以下几步：
- 数据清洗，清理掉无用的数据；
- 文本编码，将每一个单词转化为一个数字来表示；
- 将编码后的文本转化为定长表示；
- 将文本提取为特征向量进行下一步的训练。
在 TensorFlow 之中我们可以采用预处理的方式来将编码后的文本转化为定长：


In [ ]:
# 采用预处理的方式来将编码后的文本转化为定长
train_data = tf.keras.preprocessing.sequence.pad_sequences(
            train_data,
            value=0,
            padding='post',
            maxlen=10
        )


其中的各个参数的解释如下：

trian_data：我们要处理的、编码后的数据；
maxlen：将每个文本样本处理后的长度，如果原长度不足 maxlen ，那么便会使用 value 进行填充；如果原长度超过了 maxlen ，那么便会将文本截断；
value：用来填充文本的数字，一般我们使用0即可；
padding：填充的模式，post 表示填充的 value 位置在原文之后。
我们举个简单的例子，如果处理前的文本数组为：
[1, 2, 3]
当我们使用上述方式填充之后的数据就会变为：
[1, 2, 3, 0, 0 ,0, 0, 0, 0, 0]

2.2 如何将文本数组进行嵌入并提取特征向量
在 TensorFlow 之中，我们最常用的提取文本特征的网络层是：
tf.keras.layers.Embedding(vocab_size, dim),
其中 vocab_size 表示的是词汇量的总数，dim 表示特征向量的维度。
通过输入编码后的文本数组，我们可以得到该文本的特征向量（embedding vector）

3. 模型的完整表示


In [2]:
import tensorflow as tf
import numpy as np

In [ ]:
# 定义基本参数
words_num=10000
val_num=12500
EPOCHS=30
pad_max_length=256
BATCH_SIZE=64

# 获取数据
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=words_num)
word_index = tf.keras.datasets.imdb.get_word_index()

# 添加特殊字符
word_index={k:(v+3) for k,v in word_index.items()}
word_index["<pad>"]=0
word_index['<sytart>']=1
word_index['unknown']=2
word_index['unused']=3

# 数据预处理
train_data=tf.keras.preprocessing.sequence.pad_sequences(train_data,
                                                         value=0,
                                                         padding='post',
                                                        maxlen=pad_max_length)
test_data=tf.keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=pad_max_length)

# 划分训练集合与验证集合
x_val,x_train=train_data[:val_num],test_data[val_num:]
y_val,y_train=train_labels[:val_num],train_labels[val_num:]

# 模型构建
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(words_num,32),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1,activation='sinmod')
])
model.summary()

# 编译模型
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# 训练
history=model.fit(x_train,y_train,
                  epochs=EPOCHS,batch_size=BATCH_SIZE,
                  validation_data=(x_val,y_val))

# 测试
results=model.evaluate(test_data,test_labels)

print(results)





在云平台上跑出来的结果：
Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb.npz
17465344/17464789 [==============================] - 10s 1us/step
Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb_word_index.json
1646592/1641221 [==============================] - 3s 2us/step

在这里model.summary()用于输出以下信息：
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   

embedding (Embedding)        (None, None, 32)          320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        

Total params: 322,177
Trainable params: 322,177
Non-trainable params: 0

这一部分是训练过程中的结果：
Epoch 1/30
196/196 [==============================] - 61s 312ms/step - loss: 0.6349 - accuracy: 0.6977 - val_loss: 0.4827 - val_accuracy: 0.8316
Epoch 2/30
196/196 [==============================] - 128s 653ms/step - loss: 0.3499 - accuracy: 0.8702 - val_loss: 0.3144 - val_accuracy: 0.8725
Epoch 3/30
196/196 [==============================] - 109s 558ms/step - loss: 0.2362 - accuracy: 0.9108 - val_loss: 0.2892 - val_accuracy: 0.8842
Epoch 4/30
196/196 [==============================] - 103s 528ms/step - loss: 0.1821 - accuracy: 0.9342 - val_loss: 0.2941 - val_accuracy: 0.8790
Epoch 5/30
196/196 [==============================] - 50s 257ms/step - loss: 0.1416 - accuracy: 0.9526 - val_loss: 0.2968 - val_accuracy: 0.8852
Epoch 6/30
196/196 [==============================] - 45s 230ms/step - loss: 0.1098 - accuracy: 0.9662 - val_loss: 0.3142 - val_accuracy: 0.8829
Epoch 7/30
196/196 [==============================] - 39s 198ms/step - loss: 0.0883 - accuracy: 0.9756 - val_loss: 0.3449 - val_accuracy: 0.8781
Epoch 8/30
196/196 [==============================] - 91s 465ms/step - loss: 0.0678 - accuracy: 0.9841 - val_loss: 0.3687 - val_accuracy: 0.8787
Epoch 9/30
196/196 [==============================] - 102s 521ms/step - loss: 0.0523 - accuracy: 0.9884 - val_loss: 0.4041 - val_accuracy: 0.8754
Epoch 10/30
196/196 [==============================] - 125s 635ms/step - loss: 0.0399 - accuracy: 0.9932 - val_loss: 0.4376 - val_accuracy: 0.8718
Epoch 11/30
196/196 [==============================] - 80s 407ms/step - loss: 0.0297 - accuracy: 0.9955 - val_loss: 0.4725 - val_accuracy: 0.8698
Epoch 12/30
196/196 [==============================] - 44s 226ms/step - loss: 0.0229 - accuracy: 0.9968 - val_loss: 0.5156 - val_accuracy: 0.8666
Epoch 13/30
196/196 [==============================] - 42s 216ms/step - loss: 0.0185 - accuracy: 0.9973 - val_loss: 0.5382 - val_accuracy: 0.8677
Epoch 14/30
196/196 [==============================] - 93s 476ms/step - loss: 0.0135 - accuracy: 0.9982 - val_loss: 0.5784 - val_accuracy: 0.8651
Epoch 15/30
196/196 [==============================] - 104s 529ms/step - loss: 0.0104 - accuracy: 0.9990 - val_loss: 0.6044 - val_accuracy: 0.8648
Epoch 16/30
196/196 [==============================] - 99s 505ms/step - loss: 0.0076 - accuracy: 0.9995 - val_loss: 0.6342 - val_accuracy: 0.8638
Epoch 17/30
196/196 [==============================] - 91s 464ms/step - loss: 0.0058 - accuracy: 0.9996 - val_loss: 0.6630 - val_accuracy: 0.8637
Epoch 18/30
196/196 [==============================] - 42s 215ms/step - loss: 0.0047 - accuracy: 0.9998 - val_loss: 0.6909 - val_accuracy: 0.8632
Epoch 19/30
196/196 [==============================] - 42s 216ms/step - loss: 0.0035 - accuracy: 0.9999 - val_loss: 0.7158 - val_accuracy: 0.8612
Epoch 20/30
196/196 [==============================] - 63s 324ms/step - loss: 0.0031 - accuracy: 0.9998 - val_loss: 0.7429 - val_accuracy: 0.8610
Epoch 21/30
196/196 [==============================] - 94s 482ms/step - loss: 0.0023 - accuracy: 0.9999 - val_loss: 0.7655 - val_accuracy: 0.8600
Epoch 22/30
196/196 [==============================] - 99s 506ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.7877 - val_accuracy: 0.8604
Epoch 23/30
196/196 [==============================] - 111s 568ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.8082 - val_accuracy: 0.8602
Epoch 24/30
196/196 [==============================] - 59s 301ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.8303 - val_accuracy: 0.8597
Epoch 25/30
196/196 [==============================] - 41s 210ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.8497 - val_accuracy: 0.8596
Epoch 26/30
196/196 [==============================] - 42s 214ms/step - loss: 8.5455e-04 - accuracy: 1.0000 - val_loss: 0.8722 - val_accuracy: 0.8590
Epoch 27/30
196/196 [==============================] - 98s 499ms/step - loss: 7.3869e-04 - accuracy: 1.0000 - val_loss: 0.8862 - val_accuracy: 0.8593
Epoch 28/30
196/196 [==============================] - 106s 539ms/step - loss: 5.8743e-04 - accuracy: 1.0000 - val_loss: 0.9040 - val_accuracy: 0.8594
Epoch 29/30
196/196 [==============================] - 99s 503ms/step - loss: 5.0454e-04 - accuracy: 1.0000 - val_loss: 0.9229 - val_accuracy: 0.8587
Epoch 30/30
196/196 [==============================] - 91s 464ms/step - loss: 4.5580e-04 - accuracy: 1.0000 - val_loss: 0.9406 - val_accuracy: 0.8578
782/782 [==============================] - 31s 39ms/step - loss: 0.9988 - accuracy: 0.8474

这部分是最后模型评估的结果：
[0.9988135695457458, 0.8474400043487549]

对结果进行分析一波：
由结果可以看出来，模型自己的正确率到了100%，在验证集上的正确率最高为88.42%
测试集上的正确率为：84.744%
模型是否过拟合现在还分析不出来